# LSTM手写识别 [<img src="http://ogtxggxo6.bkt.clouddn.com/home.jpg?imageView2/2/h/30" align="right"/>](http://petersansan.top)
```
Author: PeterSansan
Times : 2017-4-27
```

#### 代码描述:
这个文章很好的展示了RNN网络大体的数据变化过程，RNN的输入是一个序列（这里只讨论固定长度的序列），序列的长度与timestep有关，尤其图像间不存在时序关系，图像内像素才有时序关系，所以可以考虑用一幅图构成多个序列，像这里的手写图片可以构成24*24的序列，即24组24像素长的序列，前面一个24即为timestep，后面那个24为输入层的神经元个数。上一篇的《》采用的是一组数据就更新一次参数，叫作Stochastic Gradient Descent (SGD)；也可以同时并行算多组数据，再用平均值迭代，即为Mini-batch gradient descent——小批量递度下降法。这样的技术只能后向传播有影响，又前向传播没有影响。

这个例子有几个支影响结果的超参数：
- learning_rate 学习速度
- training_iters 迭代次数
- batch_size 并列计算组数
- n_hidden 隐藏层结点数

In [1]:
%%time
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./../mnist/MNIST_data/", one_hot=True)

Extracting ./../mnist/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-labels-idx1-ubyte.gz
CPU times: user 1.1 s, sys: 326 ms, total: 1.43 s
Wall time: 1.18 s


为了有将我使用RNN网络，我们把原本28*28的图像，看成是28个序列的数据，每一个序列为一行，因此每一个数据样本我们会处理28个时间步与28个序列，这表示输入结点是28个，一次输入一个长28的序列，连续输入28个序列，再反向传播把参数更新一次。
```
To classify images using a reccurent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
```

In [2]:
# Parameters
learning_rate = 0.001 # 学习速率
training_iters = 100000 # 迭代次数，这里是数据组数
batch_size = 128 # Mini-batch 并行计算数，多少组数据更新一次参数
display_step = 10 # 多少次迭代显示一次训练精度

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print("done")

done


In [3]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    # 这个函数把原来[batch_size,n_steps,n_inputs]的数据格式转成[n_steps,batch_size,n_input]的数据格式
    # 如
    #>>> array([[[ 1,  2],
    #    [ 3,  4]],
    #   [[ 5,  6],
    #    [ 7,  8]],
    #   [[ 9, 10],
    #    [11, 12]]], dtype=int32)
    # 转化为：
    # >>> array([[ 1,  2],
    #   [ 5,  6],
    #   [ 9, 10]], dtype=int32)
    # >>> b[1].eval()
    # array([[ 3,  4],
    #   [ 7,  8],
    #   [11, 12]], dtype=int32)
    #
    x = tf.unstack(x, n_steps, 1)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    # outputs维度是[28,] , outputs[-1]维度是
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [4]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc,pp = sess.run([accuracy,pred] ,feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            print(np.shape(pp))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))


Iter 1280, Minibatch Loss= 1.852304, Training Accuracy= 0.29688
(128, 10)
Iter 2560, Minibatch Loss= 1.528903, Training Accuracy= 0.50781
(128, 10)
Iter 3840, Minibatch Loss= 1.256399, Training Accuracy= 0.56250
(128, 10)
Iter 5120, Minibatch Loss= 0.878097, Training Accuracy= 0.71094
(128, 10)
Iter 6400, Minibatch Loss= 0.789844, Training Accuracy= 0.75000
(128, 10)
Iter 7680, Minibatch Loss= 0.831269, Training Accuracy= 0.72656
(128, 10)
Iter 8960, Minibatch Loss= 0.619914, Training Accuracy= 0.78125
(128, 10)
Iter 10240, Minibatch Loss= 0.699559, Training Accuracy= 0.82031
(128, 10)
Iter 11520, Minibatch Loss= 0.692351, Training Accuracy= 0.82812
(128, 10)
Iter 12800, Minibatch Loss= 0.505655, Training Accuracy= 0.85156
(128, 10)
Iter 14080, Minibatch Loss= 0.352193, Training Accuracy= 0.89844
(128, 10)
Iter 15360, Minibatch Loss= 0.523858, Training Accuracy= 0.82812
(128, 10)
Iter 16640, Minibatch Loss= 0.260656, Training Accuracy= 0.90625
(128, 10)
Iter 17920, Minibatch Loss= 0.35

- [另一个很像的版本](http://218.192.169.151:8888/notebooks/TF_ACTION/notes/3_recurrent_network_1.ipynb)

[<img src="http://ogtxggxo6.bkt.clouddn.com/home.jpg?imageView2/2/h/30" align="left"/>](http://petersansan.top)